In [1]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tifffile
import tqdm

In [2]:
def calcualte_snr(image: np.ndarray) -> float:
    """
    Calculate the signal to noise ratio of an image
    Where SNR = mean(signal) / std(noise)
    Where the signal is the whole image pixel values

    Parameters
    ----------
    image_path : pathlib.Path
        Path to the image file

    Returns
    -------
    float
        Signal to noise ratio
    """
    p_sig = np.mean(image)
    p_noise = np.std(image)
    snr = p_sig / p_noise
    return snr

In [3]:
images_path = pathlib.Path("../../data/NF0014/zstack_images").resolve(strict=True)
# get all the tif files recursively
tif_files = list(images_path.rglob("*.tif"))
test_file = tif_files[0]

In [4]:
output_dict = {"well": [], "channel": [], "z-slice": [], "SNR": []}
for path_of_image in tqdm.tqdm(tif_files):
    well = path_of_image.stem.split("_")[0]
    channel = path_of_image.stem.split("_")[1]
    image = tifffile.imread(path_of_image)
    for z_slice in range(image.shape[0]):
        snr = calcualte_snr(image[z_slice])
        output_dict["well"].append(well)
        output_dict["channel"].append(channel)
        output_dict["z-slice"].append(z_slice)
        output_dict["SNR"].append(snr)

 69%|██████▉   | 357/515 [00:55<00:14, 10.70it/s]/tmp/ipykernel_3712600/1924429124.py:19: RuntimeWarning: invalid value encountered in scalar divide
  snr = p_sig / p_noise
100%|██████████| 515/515 [01:14<00:00,  6.92it/s]


In [5]:
df = pd.DataFrame(output_dict)
print(df.shape)
df.head()

(16850, 4)


,well,channel,z-slice,SNR
0,G8-1,405,0,2.471419
1,G8-1,405,1,1.957894
2,G8-1,405,2,1.687372
3,G8-1,405,3,1.605053
4,G8-1,405,4,1.591090


In [6]:
# save the SNR data to a csv file
pathlib.Path("../results").mkdir(exist_ok=True)
df.to_csv("../results/snr_data.csv", index=False)

Typical SNR values
* Low signal/quality confocal images (and STED): SNR = 5-10
* Average confocal image: SNR = 15-20
* High quality confocal image: SNR = > 30
* Low quality widefield image (slidescanners): SNR 5-15
* Good quality widefield image: SNR > 40
* Microscope with cooled CCD camera: SNR = 50-100 